In [2]:
!pip install requests

In [3]:
import requests #you can use the curl command as well
import datetime
from urllib.parse import urlencode

In [4]:
import base64

In [5]:
client_id = 'a0b548bd1bd64de2b3277d9dc4d12923'
client_secret = '5b9544235a6243eca69907df30149128'

In [6]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True 
    client_id = None
    client_secret = None 
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs) 
        self.client_id = client_id
        self.client_secret = client_secret
        
    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_id == None or client_secret == None:
            raise Exception ('you must set client id or client secret')
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}" 
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        }
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)

        if r.status_code not in range (200,299):
            raise Exception("Could not authenticate client.")
            #return False
        data = r.json()
        now = datetime.datetime.now() #gives you the exact current time
        access_token = data['access_token']
        expires_in = data['expires_in'] #seconds
        expires = now + datetime.timedelta(seconds=expires_in) #gives back datetime object relative to now
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        #auth.done = self.perform_auth()
        #if not auth_done:
            #raise Exception("auth failed")
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
    
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')
    
    def search(self, query, search_type='artist'): #type
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        data = urlencode({"q": query, "type":search_type.lower()})
        lookup_url = f"{endpoint}?{data}" 
        print(lookup_url)
        r = requests.get(lookup_url, headers=headers)
        if r.status_code in range (200,299):
            return r.json() 
        return {} #return empty lookup
    

In [7]:
spotify = SpotifyAPI(client_id, client_secret)

In [8]:
spotify.search("amine", search_type="track")

https://api.spotify.com/v1/search?q=amine&type=track


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=amine&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3Gm5F95VdRxW3mqCn8RPBJ'},
       'href': 'https://api.spotify.com/v1/artists/3Gm5F95VdRxW3mqCn8RPBJ',
       'id': '3Gm5F95VdRxW3mqCn8RPBJ',
       'name': 'Aminé',
       'type': 'artist',
       'uri': 'spotify:artist:3Gm5F95VdRxW3mqCn8RPBJ'}],
     'available_markets': ['AD',
      'AE',
      'AL',
      'AR',
      'AT',
      'AU',
      'BA',
      'BE',
      'BG',
      'BH',
      'BO',
      'BR',
      'CA',
      'CH',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DE',
      'DK',
      'DO',
      'DZ',
      'EC',
      'EE',
      'EG',
      'ES',
      'FI',
      'FR',
      'GB',
      'GR',
      'GT',
      'HK',
      'HN',
      'HR',
      'HU',
      'ID',
      'IE',
      'IL',
      'IN',
      'IS',
      'IT',
    

In [9]:
spotify.get_artist("3Gm5F95VdRxW3mqCn8RPBJ")

{'external_urls': {'spotify': 'https://open.spotify.com/artist/3Gm5F95VdRxW3mqCn8RPBJ'},
 'followers': {'href': None, 'total': 1427826},
 'genres': ['hip hop', 'pop rap', 'portland hip hop', 'rap'],
 'href': 'https://api.spotify.com/v1/artists/3Gm5F95VdRxW3mqCn8RPBJ',
 'id': '3Gm5F95VdRxW3mqCn8RPBJ',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/474d98b081007f98edd402e5889304eade75fe57',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/b2eb77cb075bef43f55462156bbb5d2a95cca9e8',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ea6ae0d29cf569a6828e5153e6e0ada26c6d1bba',
   'width': 160}],
 'name': 'Aminé',
 'popularity': 79,
 'type': 'artist',
 'uri': 'spotify:artist:3Gm5F95VdRxW3mqCn8RPBJ'}

In [10]:
spotify.get_album("3Gm5F95VdRxW3mqCn8RPBJ")

{}